In [59]:
import os
import sys
import json
import pickle
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from pprint import pprint

sys.path.append("../")
from utils.parse_arxiv import parse_title

from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

def sort_dict_by_key(d):
    return {k: v for k, v in sorted(d.items(), key=lambda item: item[0])}

def sort_dict_by_val(d):
    return {k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse=True)}

[nltk_data] Downloading package punkt to /home/zyang37/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [67]:
file = open('../data/cnn_news/filtered_dataCNN.pickle', 'rb')
data = pickle.load(file)
file.close()


clean_cols = ['Headline', 'Description', 'Second headline', 'Article text', 'Keywords']

print(len(data))
for c in clean_cols:
    data[c] = data[c].map(parse_title).values
    data.drop_duplicates(subset=[c], inplace=True)

data.drop_duplicates(subset=['Url'], inplace=True)
print(len(data))

data.head(1)

26980
26921


,Index,Author,Date published,Category,Section,Url,Headline,Description,Keywords,Second headline,Article text,author_freq,cat_freq,sec_freq,keys_freq,Article_len,Headline_len,len_ratio,date_freq
1,1,"Stephanie Bailey, CNN",2021-05-12,news,world,https://www.cnn.com/2021/05/12/world/ironhand-...,Bioservo's robotic 'Ironhand' could protect fa...,Working in a factory can mean doing the same t...,"world, Bioservo's robotic 'Ironhand' could pro...",A robotic 'Ironhand' could protect factory wor...,(CNN)Working in a factory or warehouse can mea...,13,18011,614,0,5815,79,0.013586,10


In [68]:
text = data.iloc[0]['Article text']
len(text)

5810

In [69]:
sentences = sent_tokenize(text)

In [70]:
def token_limited_sentences(text, max_num_tokens=300):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)

    # Initialize an empty list to store processed sentences
    processed_sentences = []
    current_sentence = ""
    for sentence in sentences:
        # Tokenize the sentence into words
        tokenized_sentence = word_tokenize(sentence)

        # Check if adding this sentence exceeds the token limit
        if len(word_tokenize(current_sentence)) + len(tokenized_sentence) <= max_num_tokens:
            current_sentence += " " + sentence
        else:
            # Add the current sentence to the list if it's not empty
            if current_sentence:
                processed_sentences.append(current_sentence.strip())
            # Start a new sentence
            current_sentence = sentence

    # Add the last sentence if it's not empty
    if current_sentence:
        processed_sentences.append(current_sentence.strip())

    return processed_sentences

In [75]:
texts_chunks = token_limited_sentences(text, 300)
for i, s in enumerate(texts_chunks):
    # print(i, len(s))
    tag = "p{}_{}_".format(i+1, len(texts_chunks))
    print(tag)
    print(s)
    print()

p1_4_
(CNN)Working in a factory or warehouse can mean doing the same task over and over, and that repetition can lead to chronic injury. Now, a battery-powered glove could help workers by taking some of the strain.The "Ironhand" glove strengthens the wearer's grip, meaning they don't have to use as much force to perform repetitive manual tasks. Its developer, Bioservo, says it can increase the wearer's hand strength by 20%. The Swedish company describes the system as a "soft exoskeleton." Exoskeletons are an external device that supports and protects the body, typically increasing strength and endurance. Most have a rigid structure, but the Ironhand is soft, like a regular glove. Photos: The robots running our warehousesRobots are an increasingly familiar presence in warehouses. At the south-east London warehouse run by British online supermarket Ocado, 3,000 robots fulfill shopping orders. When an order is sent to the warehouse, the bots spring to life and head towards the container t

In [72]:
max_chunk_size = 300
new_df = pd.DataFrame()

for index, row in data.iterrows():
    row_dict = row.to_dict()
    # pprint(row_dict)
    texts_chunks = token_limited_sentences(text, 300)
    for i, s in enumerate(texts_chunks):
        # print(i, len(s))
        chunk_id = "p{}_{}_".format(i+1, len(texts_chunks))
        print(tag)
        print(s)
        print()
    # df = df.append(new_row, ignore_index=True)
    break

p1_4
(CNN)Working in a factory or warehouse can mean doing the same task over and over, and that repetition can lead to chronic injury. Now, a battery-powered glove could help workers by taking some of the strain.The "Ironhand" glove strengthens the wearer's grip, meaning they don't have to use as much force to perform repetitive manual tasks. Its developer, Bioservo, says it can increase the wearer's hand strength by 20%. The Swedish company describes the system as a "soft exoskeleton." Exoskeletons are an external device that supports and protects the body, typically increasing strength and endurance. Most have a rigid structure, but the Ironhand is soft, like a regular glove. Photos: The robots running our warehousesRobots are an increasingly familiar presence in warehouses. At the south-east London warehouse run by British online supermarket Ocado, 3,000 robots fulfill shopping orders. When an order is sent to the warehouse, the bots spring to life and head towards the container th

In [74]:
# row.to_dict()

# collection.add(
#     embeddings=[[1.2, 2.3, 4.5], [6.7, 8.2, 9.2]],
#     documents=["This is a document", "This is another document"],
#     metadatas=[{"source": "my_source"}, {"source": "my_source"}],
#     ids=["id1", "id2"]
# )